This notebook is designed to generate the map statistics presented in the gulf sturgeon substrate manuscript.

In [1]:
#########
# Imports
import sys, os
import numpy as np
import geopandas as gpd
import pandas as pd
from glob import glob

pd.option_context('display.max_rows', None, 'display.max_columns', None)

Get the paths to the shapefiles. Shapefiles with `proc` in their name have been post-processed in ArcGIS.

In [ ]:
shpTopDir = r'D:\redbo_science\projects\GulfSturgeonProject_2025\ProcessedData\Substrate_Summaries\02_Substrate_Shps_Mosaic_Transects'
shpTopDir = os.path.normpath(shpTopDir)
# # Get EGN shps
# egnShpFiles = glob(os.path.join(shpTopDir, 'EGN', '*proc.shp'))
# print(egnShpFiles)

# Get Raw Shps
rawShpFiles = glob(os.path.join(shpTopDir, 'Raw', '*proc.shp'))
print(rawShpFiles)

['E:\\SynologyDrive\\GulfSturgeonProject\\SSS_Data_Processed\\Substrate_Summaries\\02_Substrate_Shps_Mosaic_Transects\\EGN\\BCH_substrate_shps_mosaic_postproc.shp', 'E:\\SynologyDrive\\GulfSturgeonProject\\SSS_Data_Processed\\Substrate_Summaries\\02_Substrate_Shps_Mosaic_Transects\\EGN\\BOU_substrate_shps_mosaic_postproc.shp', 'E:\\SynologyDrive\\GulfSturgeonProject\\SSS_Data_Processed\\Substrate_Summaries\\02_Substrate_Shps_Mosaic_Transects\\EGN\\CHI_substrate_shps_mosaic_postproc.shp', 'E:\\SynologyDrive\\GulfSturgeonProject\\SSS_Data_Processed\\Substrate_Summaries\\02_Substrate_Shps_Mosaic_Transects\\EGN\\CHU_substrate_shps_mosaic_postproc.shp', 'E:\\SynologyDrive\\GulfSturgeonProject\\SSS_Data_Processed\\Substrate_Summaries\\02_Substrate_Shps_Mosaic_Transects\\EGN\\LEA_substrate_shps_mosaic_postproc.shp', 'E:\\SynologyDrive\\GulfSturgeonProject\\SSS_Data_Processed\\Substrate_Summaries\\02_Substrate_Shps_Mosaic_Transects\\EGN\\PAS_substrate_shps_mosaic_postproc.shp', 'E:\\SynologyDr

# Prep Dataframes

In [38]:
def getBasin(river):
    prl = 'Pearl Basin'
    pas = 'Pascagoula Basin'

    if river == 'BCH':
        basin = prl
    elif river == 'PRL':
        basin = prl
    elif river == 'BOU':
        basin = pas
    elif river == 'LEA':
        basin = pas
    elif river == 'PAS':
        basin = pas
    elif river == 'CHI':
        basin = pas
    elif river == 'CHU':
        basin = pas
    else:
        basin = 'UNKNOWN'
    return basin

In [40]:
# Function to open dataframe, get river code from name, and populate attributes
def getShp(f):
    river = os.path.basename(f).split('_')[0]
    model = os.path.basename(os.path.dirname(f))
    basin = getBasin(river)
    
    df = gpd.read_file(f)
    df['river'] = river
    df['model'] = model
    df['basin'] = basin

    # Calculate area in hectars
    df['Area_m'] = df.geometry.area
    df['Area_ha'] = df['Area_m']*0.0001

    return df

In [ ]:
# # Get EGN shapefiles to geodataframe
# for i, f in enumerate(egnShpFiles):
#     df = getShp(f)
#     if i == 0:
#         egnAll = df
#     else:
#         egnAll = pd.concat([egnAll, df], ignore_index=True)

# print(egnAll)

       Substrate          Name      Area_m  ORIG_FID  Shape_Leng  Shape_Area  \
0            1.0  Fines Ripple   73.346198         1   39.931794   73.346198   
1            1.0  Fines Ripple  665.473590         4  250.726137  665.473590   
2            1.0  Fines Ripple   64.828920         5   34.678850   64.828920   
3            1.0  Fines Ripple   50.861519         9   34.894188   50.861519   
4            1.0  Fines Ripple   59.763020        10   44.072426   59.763020   
...          ...           ...         ...       ...         ...         ...   
41429        6.0         Other   52.920486     21561   65.187730   52.920486   
41430        6.0         Other   35.028880     21562   60.113561   35.028880   
41431        6.0         Other   39.185818     21563   75.556242   39.185818   
41432        6.0         Other   41.075668     21565   59.459670   41.075668   
41433        6.0         Other   37.684611     21567   45.647350   37.684611   

                                       

In [42]:
# Get raw shapefiles to gdf
for i, f in enumerate(rawShpFiles):
    df = getShp(f)
    if i == 0:
        rawAll = df
    else:
        rawAll = pd.concat([rawAll, df], ignore_index=True)

print(rawAll)

       Substrate          Name      Area_m  ORIG_FID  Shape_Leng  Shape_Area  \
0            1.0  Fines Ripple   33.146532         1   33.521692   33.146532   
1            1.0  Fines Ripple  936.237783         4  273.382980  936.237783   
2            1.0  Fines Ripple   37.598220         5   59.747814   37.598220   
3            1.0  Fines Ripple  159.851689         9   68.702608  159.851689   
4            1.0  Fines Ripple   34.731836        15   26.603827   34.731836   
...          ...           ...         ...       ...         ...         ...   
39335        6.0         Other   58.050255     21370   48.066846   58.050255   
39336        6.0         Other   51.665556     21372   95.006057   51.665556   
39337        6.0         Other   28.808853     21376   67.508822   28.808853   
39338        6.0         Other  114.494662     21378  189.667690  114.494662   
39339        6.0         Other   29.376620     21379   34.995228   29.376620   

                                       

# Find the total mapped area for the study. 
Use the Raw shapefile

In [56]:
df = rawAll

In [60]:
# Total mapped area
totalArea = np.sum(df['Area_ha'].to_numpy())
print('Total Mapped Area:', np.around(totalArea, 0))

Total Mapped Area: 7425.0


In [62]:
# Pascagoula basin total area
pasTotalArea = np.sum(df.loc[df['basin'] == 'Pascagoula Basin', 'Area_ha'].to_numpy())
print('Pascagoula Mapped Area:', np.around(pasTotalArea, 0))

Pascagoula Mapped Area: 3262.0


In [63]:
# Pearl basin total area
prlTotalArea = np.sum(df.loc[df['basin'] == 'Pearl Basin', 'Area_ha'].to_numpy())
print('Pearl Mapped Area:', np.around(prlTotalArea, 0))

Pearl Mapped Area 4163.0


In [66]:
# Find each rivers mapped area
riv = df['river'].unique()
rivAreaTotal = {}
for r in riv:
    t = np.sum(df.loc[df['river'] == r, 'Area_ha'].to_numpy())
    rivAreaTotal[r] = np.around(t, 0)

for k, v in rivAreaTotal.items():
    print(k, 'Mapped Area:', v)


BCH Mapped Area: 569.0
BOU Mapped Area: 115.0
CHI Mapped Area: 1421.0
CHU Mapped Area: 89.0
LEA Mapped Area: 1159.0
PAS Mapped Area: 478.0
PRL Mapped Area: 3595.0


# Get Polygon Counts and Proportions
Work on Raw shapefile first

In [145]:
df = rawAll

In [146]:
shpTotal = pd.DataFrame()
rivers = df['river'].unique()
substrate = df['Name'].unique()

for r in rivers:
    sub = {}
    for s in substrate:
        sub['river'] = [r]
        sub['substrate'] = [s]
        dfSlice = df.loc[(df['river'] == r) & (df['Name'] == s), ['Area_m']]

        sub['total_poly'] = [len(dfSlice)]
        sub['min_poly_m'] = [np.around(np.min(dfSlice), 1)]
        sub['max_poly_m'] = [np.around(np.max(dfSlice), 1)]
        sub['total_poly_m'] = [np.around(np.sum(dfSlice['Area_m']), 1)]
        
        sub = pd.DataFrame.from_dict(sub)
        shpTotal = pd.concat([shpTotal, sub], ignore_index=True)

    
print(shpTotal)       


   river       substrate  total_poly  min_poly_m  max_poly_m  total_poly_m
0    BCH    Fines Ripple         502        28.3    248236.1     2944289.3
1    BCH      Fines Flat        1478        28.0     87160.8     2387136.1
2    BCH  Cobble Boulder         184        28.4      2187.9       33248.4
3    BCH     Hard Bottom         242        28.7      3840.9       84291.0
4    BCH            Wood        1167        28.5      2595.8      194088.4
5    BCH           Other         591        28.0       382.1       42993.1
6    BOU    Fines Ripple          21        28.5    233683.5      581552.4
7    BOU      Fines Flat         442        28.6     59356.0      436855.8
8    BOU  Cobble Boulder          81        31.0      1072.6       14333.2
9    BOU     Hard Bottom         144        28.5      9478.8       78623.0
10   BOU            Wood         234        28.9       976.9       33250.9
11   BOU           Other          70        29.2       205.7        4310.7
12   CHI    Fines Ripple 

In [147]:
print('Raw')
print('Model Polygon Summary')
df = shpTotal

t = np.sum(df['total_poly'].to_numpy())
min = np.min(df['min_poly_m'].to_numpy())
max = np.max(df['max_poly_m'].to_numpy())

print('Total Polygons:', t, '\tMin Area:', min, '\tMax Area:', max, '\n\n')

shpProp = pd.DataFrame()
rivers = df['river'].unique()
for r in rivers:
    dfSlice = df.loc[df['river'] == r]

    t = np.sum(dfSlice['total_poly'].to_numpy())
    min = np.min(dfSlice['min_poly_m'].to_numpy())
    max = np.max(dfSlice['max_poly_m'].to_numpy())

    print(r, '\tTotal Polygons:', t, '\tMin Area:', min, '\tMax Area:', max)

    dfSlice = dfSlice.pivot(index='river', columns=['substrate'])['total_poly_m']
    shpProp = pd.concat([shpProp, dfSlice])

    print('\n\n')

substrates = shpProp.columns
shpProp['total_m'] = shpProp.sum(axis=1)

for s in substrates:
    shpProp[s] = np.around((shpProp[s]/shpProp['total_m'])*100, 1)
shpProp['Soft'] = shpProp['Fines Flat'] + shpProp['Fines Ripple']
shpProp['Hard'] = shpProp['Cobble Boulder'] + shpProp['Hard Bottom']
print(shpProp)

rawFR = shpProp['Fines Ripple']
rawFF = shpProp['Fines Flat']
rawCB = shpProp['Cobble Boulder']
rawHB = shpProp['Hard Bottom']
rawWD = shpProp['Wood']
rawSoft = shpProp['Soft']
rawHard = shpProp['Hard']


Raw
Model Polygon Summary
Total Polygons: 39340 	Min Area: 28.0 	Max Area: 3845957.9 


BCH 	Total Polygons: 4164 	Min Area: 28.0 	Max Area: 248236.1



BOU 	Total Polygons: 992 	Min Area: 28.5 	Max Area: 233683.5



CHI 	Total Polygons: 8229 	Min Area: 28.0 	Max Area: 1945386.2



CHU 	Total Polygons: 870 	Min Area: 28.1 	Max Area: 132317.9



LEA 	Total Polygons: 5708 	Min Area: 28.0 	Max Area: 1135129.3



PAS 	Total Polygons: 1631 	Min Area: 28.1 	Max Area: 1890846.4



PRL 	Total Polygons: 17746 	Min Area: 28.0 	Max Area: 3845957.9



substrate  Cobble Boulder  Fines Flat  Fines Ripple  Hard Bottom  Other  Wood  \
river                                                                           
BCH                   0.6        42.0          51.8          1.5    0.8   3.4   
BOU                   1.2        38.0          50.6          6.8    0.4   2.9   
CHI                   0.5        19.1          72.1          4.6    1.0   2.6   
CHU                   1.0        29.4          19

Now do EGN

In [151]:
df = egnAll

In [152]:
shpTotal = pd.DataFrame()
rivers = df['river'].unique()
substrate = df['Name'].unique()

for r in rivers:
    sub = {}
    for s in substrate:
        sub['river'] = [r]
        sub['substrate'] = [s]
        dfSlice = df.loc[(df['river'] == r) & (df['Name'] == s), ['Area_m']]

        sub['total_poly'] = [len(dfSlice)]
        sub['min_poly_m'] = [np.around(np.min(dfSlice), 1)]
        sub['max_poly_m'] = [np.around(np.max(dfSlice), 1)]
        sub['total_poly_m'] = [np.around(np.sum(dfSlice['Area_m']), 1)]
        
        sub = pd.DataFrame.from_dict(sub)
        shpTotal = pd.concat([shpTotal, sub], ignore_index=True)

shpTotal = shpTotal.fillna(0)
print(shpTotal)   

   river       substrate  total_poly  min_poly_m  max_poly_m  total_poly_m
0    BCH    Fines Ripple         644        28.6    239637.6     2652154.7
1    BCH      Fines Flat        1583        28.0     82506.1     2672881.4
2    BCH  Cobble Boulder         264        29.5      2019.4       44536.8
3    BCH     Hard Bottom         235        28.4      2407.2       56993.8
4    BCH            Wood        1333        28.6      3743.5      261013.9
5    BCH           Other          60        28.0       340.5        4043.0
6    BOU    Fines Ripple          60        31.3    107950.2      549463.1
7    BOU      Fines Flat         428        28.0     60761.9      499526.2
8    BOU  Cobble Boulder          67        28.1       805.4        9002.1
9    BOU     Hard Bottom         130        30.4      8704.4       52907.2
10   BOU            Wood         246        28.4       919.3       36800.8
11   BOU           Other           1        35.4        35.4          35.4
12   CHI    Fines Ripple 

In [153]:
print('EGN')
print('Model Polygon Summary')
df = shpTotal

t = np.sum(df['total_poly'].to_numpy())
min = np.min(df['min_poly_m'].to_numpy())
max = np.max(df['max_poly_m'].to_numpy())

print('Total Polygons:', t, '\tMin Area:', min, '\tMax Area:', max, '\n\n')

shpProp = pd.DataFrame()
rivers = df['river'].unique()
for r in rivers:
    dfSlice = df.loc[df['river'] == r]

    t = np.sum(dfSlice['total_poly'].to_numpy())
    min = np.min(dfSlice['min_poly_m'].to_numpy())
    max = np.max(dfSlice['max_poly_m'].to_numpy())

    print(r, '\tTotal Polygons:', t, '\tMin Area:', min, '\tMax Area:', max)

    dfSlice = dfSlice.pivot(index='river', columns=['substrate'])['total_poly_m']
    shpProp = pd.concat([shpProp, dfSlice])

    print('\n\n')

substrates = shpProp.columns
shpProp['total_m'] = shpProp.sum(axis=1)

for s in substrates:
    shpProp[s] = np.around((shpProp[s]/shpProp['total_m'])*100, 1)
shpProp['Soft'] = shpProp['Fines Flat'] + shpProp['Fines Ripple']
shpProp['Hard'] = shpProp['Cobble Boulder'] + shpProp['Hard Bottom']
print(shpProp)

egnFR = shpProp['Fines Ripple']
egnFF = shpProp['Fines Flat']
egnCB = shpProp['Cobble Boulder']
egnHB = shpProp['Hard Bottom']
egnWD = shpProp['Wood']
egnSoft = shpProp['Soft']
egnHard = shpProp['Hard']

EGN
Model Polygon Summary
Total Polygons: 41434 	Min Area: 0.0 	Max Area: 1964414.8 


BCH 	Total Polygons: 4119 	Min Area: 28.0 	Max Area: 239637.6



BOU 	Total Polygons: 932 	Min Area: 28.0 	Max Area: 107950.2



CHI 	Total Polygons: 8611 	Min Area: 28.0 	Max Area: 765564.1



CHU 	Total Polygons: 792 	Min Area: 0.0 	Max Area: 123872.0



LEA 	Total Polygons: 6609 	Min Area: 28.1 	Max Area: 500647.6



PAS 	Total Polygons: 2374 	Min Area: 28.1 	Max Area: 1147217.0



PRL 	Total Polygons: 17997 	Min Area: 28.0 	Max Area: 1964414.8



substrate  Cobble Boulder  Fines Flat  Fines Ripple  Hard Bottom  Other  Wood  \
river                                                                           
BCH                   0.8        47.0          46.6          1.0    0.1   4.6   
BOU                   0.8        43.5          47.9          4.6    0.0   3.2   
CHI                   1.1        30.6          61.0          3.1    0.6   3.6   
CHU                   0.3        65.1          16.3  

Overall proportion difference per river per substrate

In [159]:
c = ['Fines Ripple', 'Fines Flat', 'Cobble Boulder', 'Hard Bottom', 'Wood', 'Soft', 'Hard']
r = [rawFR, rawFF, rawCB, rawHB, rawWD, rawSoft, rawHard]
e = [egnFR, egnFF, egnCB, egnHB, egnWD, egnSoft, egnHard]

for sub, raw, egn in zip(c, r, e):
    d = pd.concat([raw, egn], axis=1)
    d['dif'] = raw-egn
    print(sub)
    print(d)
    print('\n\n')

Fines Ripple
       Fines Ripple  Fines Ripple   dif
river                                  
BCH            51.8          46.6   5.2
BOU            50.6          47.9   2.7
CHI            72.1          61.0  11.1
CHU            19.5          16.3   3.2
LEA            76.6          68.8   7.8
PAS            81.6          73.6   8.0
PRL            52.9          48.1   4.8



Fines Flat
       Fines Flat  Fines Flat   dif
river                              
BCH          42.0        47.0  -5.0
BOU          38.0        43.5  -5.5
CHI          19.1        30.6 -11.5
CHU          29.4        65.1 -35.7
LEA          16.2        23.3  -7.1
PAS          14.8        16.9  -2.1
PRL          38.9        44.6  -5.7



Cobble Boulder
       Cobble Boulder  Cobble Boulder  dif
river                                     
BCH               0.6             0.8 -0.2
BOU               1.2             0.8  0.4
CHI               0.5             1.1 -0.6
CHU               1.0             0.3  0.7
LEA          